In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import datetime
warnings.filterwarnings("ignore")
import gc
# Any results you write to the current directory are saved as output.

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train = pd.read_csv("../input/train.csv", parse_dates=["first_active_month"])
test = pd.read_csv("../input/test.csv", parse_dates=["first_active_month"])

In [ ]:
new_m_trans=reduce_mem_usage(pd.read_csv("../input/new_merchant_transactions.csv"))
hist_trans=reduce_mem_usage(pd.read_csv("../input/historical_transactions.csv"))

In [ ]:
train.head()

In [ ]:
new_m_trans.head(10)

In [ ]:
train.head()

In [ ]:
plt.figure(figsize=(12,8))
sns.distplot(train["target"].values,kde=True)
plt.xlabel("score")
plt.title("histogram of Loyality score")
plt.show()

It shows clealy the data is not skewd and shows few outliers far away from the normal distribution. Its go and see what percentage of target is an outlier.

In [ ]:
train[train["target"]<-30].shape[0]/train.shape[0] * 100

It shows around 1.09 percent of data is far away from the normal distribution. May be it will play an role in modeling. now lets hgo and look at the box plots.

In [ ]:
plt.figure(figsize=(10,8))
import seaborn as sns 
sns.boxplot(train["target"].values)

Lets look the violin plot so that we an get a good view of the outliers.

In [ ]:
plt.figure(figsize=(10,8))
sns.set_style("whitegrid")
sns.violinplot(train["target"].values)

we can see clearly a small bump after -30 which shows clearly the outliers. Lets look the feature1,2 & 3.

In [ ]:
f, (ax1,ax2,ax3) = plt.subplots(ncols=3, figsize=(17, 6), sharex=True)
sns.despine(left=True)

sns.distplot(train["feature_1"].values,kde=False,ax=ax1)
sns.distplot(train["feature_2"].values,kde=False,ax=ax2)
sns.distplot(train["feature_3"].values,kde=False,ax=ax3)



This shows that the Feature_1,2,3 are the discreate values of (0,1,2,3,4,5,) may be these are the categorical values.

In [ ]:
new_m_trans.isnull().sum()

In [ ]:
new_m_trans.category_3.value_counts(),new_m_trans.category_2.value_counts()

In [ ]:
dic=pd.read_excel("../input/Data_Dictionary.xlsx")
dic

In [ ]:
merchants=pd.read_csv("../input/merchants.csv")
merchants.head()

In [ ]:
new_m_trans.installments.value_counts()


In [ ]:
train=pd.get_dummies(train,columns=["feature_1","feature_2"])
test=pd.get_dummies(test,columns=["feature_1","feature_2"])

In [ ]:
train.head()

In [ ]:
for df in [new_m_trans,hist_trans]:
    df["category_2"].fillna(df["category_2"].mode(),inplace=True,)
    df["category_3"].fillna("A",inplace=True)
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)

In [ ]:
hist_trans.installments.replace(999,0,inplace=True)
hist_trans.installments.replace(-1,0,inplace=True)
new_m_trans.installments.replace(999,0,inplace=True)
new_m_trans.installments.replace(-1,0,inplace=True)

In [ ]:
for df in [hist_trans,new_m_trans]:
    df["purchase_date"]=pd.to_datetime(df["purchase_date"])
    df["year"]=df["purchase_date"].dt.year
    df["weekofyear"]=df["purchase_date"].dt.weekofyear
    df["month"]=df["purchase_date"].dt.month
    df["dayofweek"]=df["purchase_date"].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df["hour"]=df["purchase_date"].dt.hour
    df["authorized_flag"]=df["authorized_flag"].map({"Y":1,"N":0})
    df["category_1"]=df["category_1"].map({"Y":1,"N":0})
    df["month_diff"]=((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df["month_diff"]+= df['month_lag']

In [ ]:
hist_trans = pd.get_dummies(hist_trans, columns=['category_2', 'category_3'])
new_m_trans=pd.get_dummies(new_m_trans, columns=["category_2","category_3"])

In [ ]:
hist_trans.head()

In [ ]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]


In [ ]:
aggs = {}
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id','state_id']:
    aggs[col] = ['nunique']

aggs['purchase_amount'] = ['sum','max','min','mean','var']
aggs['installments'] = ['sum','max','min','mean','var']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ["min","max",'mean']
aggs['authorized_flag'] = ['sum', 'mean']
aggs['weekend'] = ['sum', 'mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']
aggs['category_1']= ['sum','mean']
aggs['category_2_1.0']= ['mean','sum']
aggs['category_2_2.0']= ['mean','sum']
aggs['category_2_3.0']= ['mean','sum']
aggs['category_2_4.0']=['mean','sum']
aggs['category_2_5.0']= ['mean','sum']
aggs['category_3_A']= ['mean','sum']
aggs['category_3_B']= ['mean','sum']
aggs['category_3_C']= ['mean','sum']



new_columns = get_new_columns('hist',aggs)
hist_trans_group = hist_trans.groupby('card_id').agg(aggs)
hist_trans_group.columns = new_columns
hist_trans_group.reset_index(drop=False,inplace=True)
hist_trans_group['hist_purchase_date_diff'] = (hist_trans_group['hist_purchase_date_max'] - hist_trans_group['hist_purchase_date_min']).dt.days
hist_trans_group['hist_purchase_date_average'] = hist_trans_group['hist_purchase_date_diff']/hist_trans_group['hist_card_id_size']
hist_trans_group['hist_purchase_date_uptonow'] = (datetime.datetime.today() - hist_trans_group['hist_purchase_date_max']).dt.days
train = train.merge(hist_trans_group,on='card_id',how='left')
test = test.merge(hist_trans_group,on='card_id',how='left')
del hist_trans_group;gc.collect();gc.collect()

In [ ]:
aggs = {}
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id','state_id']:
    aggs[col] = ['nunique']
aggs['purchase_amount'] = ['sum','max','min','mean','var']
aggs['installments'] = ['sum','max','min','mean','var']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ["min","max",'mean']
aggs['authorized_flag'] = ['sum', 'mean']
aggs['weekend'] = ['sum', 'mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']
aggs['category_1']= ['sum','mean']
aggs['category_2_1.0']= ['mean','sum']
aggs['category_2_2.0']= ['mean','sum']
aggs['category_2_3.0']= ['mean','sum']
aggs['category_2_4.0']=['mean','sum']
aggs['category_2_5.0']= ['mean','sum']
aggs['category_3_A']= ['mean','sum']
aggs['category_3_B']= ['mean','sum']
aggs['category_3_C']= ['mean','sum']


    
new_columns = get_new_columns('new_hist',aggs)
hist_trans_group = new_m_trans.groupby('card_id').agg(aggs)
hist_trans_group.columns = new_columns
hist_trans_group.reset_index(drop=False,inplace=True)
hist_trans_group['new_hist_purchase_date_diff'] = (hist_trans_group['new_hist_purchase_date_max'] - hist_trans_group['new_hist_purchase_date_min']).dt.days
hist_trans_group['new_hist_purchase_date_average'] = hist_trans_group['new_hist_purchase_date_diff']/hist_trans_group['new_hist_card_id_size']
hist_trans_group['new_hist_purchase_date_uptonow'] = (datetime.datetime.today() - hist_trans_group['new_hist_purchase_date_max']).dt.days
train = train.merge(hist_trans_group,on='card_id',how='left')
test = test.merge(hist_trans_group,on='card_id',how='left')
del hist_trans_group;gc.collect();gc.collect()

In [ ]:

del hist_trans;gc.collect()
del new_m_trans;gc.collect()
train.head(5)


In [ ]:
train['outliers'] = 0
train.loc[train['target'] < -30, 'outliers'] = 1
train['outliers'].value_counts()

In [ ]:

for df in [train,test]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_hist_first_buy'] = (df['new_hist_purchase_date_min'] - df['first_active_month']).dt.days
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_hist_purchase_date_max',\
                     'new_hist_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9
    df['card_id_total'] = df['new_hist_card_id_size']+df['hist_card_id_size']
    df['purchase_amount_total'] = df['new_hist_purchase_amount_sum']+df['hist_purchase_amount_sum']




In [ ]:

for f in ['feature_1_1',"feature_1_2","feature_1_3","feature_1_4","feature_1_5",'feature_2_1',"feature_2_2","feature_2_3"]:
    order_label = train.groupby([f])['outliers'].mean()
    train[f] = train[f].map(order_label)
    test[f] = test[f].map(order_label)

In [ ]:
train_columns = [c for c in train.columns if c not in ['card_id', 'first_active_month','target','outliers']]
target = train['target']
del train['target']

In [ ]:

import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
param = {'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 4,
         "random_state": 4590}

oof = np.zeros(len(train))
predictions = np.zeros(len(test))
#start = time.time()

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=15)

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train,train['outliers'].values)):    
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][train_columns], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train.iloc[val_idx][train_columns], label=target.iloc[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
    oof[val_idx] = clf.predict(train.iloc[val_idx][train_columns], num_iteration=clf.best_iteration)
    
    predictions += clf.predict(test[train_columns], num_iteration=clf.best_iteration) / folds.n_splits



In [ ]:
from sklearn.metrics import mean_squared_error
val_score = np.sqrt(mean_squared_error(target, oof))
val_score

In [ ]:
sample_submission = pd.read_csv('../input/sample_submission.csv')
sample_submission['target'] = predictions
sample_submission.to_csv('predict.csv', index=False)